## Import

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import sys
import json
from datetime import datetime
from tqdm import tqdm
import pytz
%matplotlib inline

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

## Load Data

In [2]:
# Training and Testing directories
training_dir = os.path.join("Datasets", "Training")
testing_dir = os.path.join("Datasets", "Testing")
if not os.path.isdir(training_dir):
    raise Exception("ERROR: training dataset not found")
if not os.path.isdir(testing_dir):
    raise Exception("ERROR: testing dataset not found")

In [3]:
# iterate over all hashtag files 
for root, dirs, files in os.walk(training_dir, topdown=False):
    for file in files:
        print(os.path.join(root, file))

Datasets/Training/tweets_#gohawks.txt
Datasets/Training/tweets_#gopatriots.txt
Datasets/Training/tweets_#nfl.txt
Datasets/Training/tweets_#patriots.txt
Datasets/Training/tweets_#sb49.txt
Datasets/Training/tweets_#superbowl.txt


In [4]:
# Initialize dictionaries and Unix times for Feb 1, 8 am and Feb 1, 8 pm. 
# Dictionary keys: hashtag.
# Dictionary values: [time of tweet (Unix), number of retweets for tweet, number of followers for tweeter]
# Each row in dictionary value is an individual tweet.

hashtag_dict_before = {}
hashtag_dict_during = {}
hashtag_dict_after = {}
start_unix_time = 1422806400 # 8 am, Feb 1, PST
end_unix_time = 1422849600 # 8 pm, Feb 1, PST
pst_tz = pytz.timezone('America/Los_Angeles')


In [5]:
""" Parse files to get necessary data """

for root, dirs, files in os.walk(training_dir, topdown=False):
    for file in files:
        filename = os.path.splitext(file)[0].replace('tweets_#', '')
        print('Parsing {}...'.format(filename))
        
        hashtag_dict_before[filename] = []
        hashtag_dict_during[filename] = []
        hashtag_dict_after[filename] = []
        
        # open the file and read all lines:
        with open(os.path.join(root, file), "r", encoding="utf-8") as hashtag:
            # read line-by-line
            for line in hashtag:
                json_obj = json.loads(line)
                
                # get desired statistics
                citation_date = json_obj['citation_date'] # Unix time
                num_retweets = json_obj['metrics']['citations']['total'] # Number of retweets for this tweet
                num_followers = json_obj['author']['followers'] # Number of followers for tweeter
                
                # Check when tweet was made and add it to corresponding dictionary
                if citation_date < start_unix_time:
                    hashtag_dict_before[filename].append([citation_date, num_retweets, num_followers])
                elif citation_date > end_unix_time:
                    hashtag_dict_after[filename].append([citation_date, num_retweets, num_followers])
                else:
                    hashtag_dict_during[filename].append([citation_date, num_retweets, num_followers])
    print('done')

Parsing gohawks...
Parsing gopatriots...
Parsing nfl...
Parsing patriots...
Parsing sb49...
Parsing superbowl...
done


## Organize Data

##### Variables:
key = one of the hashtags

data_hashtag_before[key] = data before 2/1 8am, split into 1-hour windows (separated by hashtag) <br \>
data_hashtag_during[key] = data between 2/1 8am and 8pm, split into 5-min windows (separated by hashtag) <br \>
data_hashtag_after[key] = data after 2/1 8pm, split into 1-hour windows (separated by hashtag) <br \>

data_aggregate_before = data before 2/1 8am, split into 1-hour windows (all hashtags combined) <br \>
data_aggregate_during = data between 2/1 8am and 8pm, split into 5-min windows (all hashtags combined) <br \>
data_aggregate_after = data after 2/1 8pm, split into 1-hour windows (all hashtags combined) <br \>

data_hashtag_all[key] = all data, split into 1-hour windows (separated by hashtag) <br \>

data_all = all data, split into 1-hour windows (all hashtags combined) <br \>

In [6]:
# Explicitly list hashtags. 
# Convert each value in dictionary to numpy arrays.

hashtags = ['gohawks', 'gopatriots', 'nfl', 'patriots', 'sb49', 'superbowl']

for key in hashtags:
    hashtag_dict_before[key] = np.array(hashtag_dict_before[key])
    hashtag_dict_during[key] = np.array(hashtag_dict_during[key])
    hashtag_dict_after[key] = np.array(hashtag_dict_after[key])

In [7]:
# Find how many time windows there are

ftt = int(np.min([np.min(hashtag_dict_before[key][:,0]) for key in hashtags])) # first tweet time
ltt = int(np.max([np.max(hashtag_dict_after[key][:,0]) for key in hashtags])) # last tweet time

num_windows_before = int(np.max([((start_unix_time - ftt) // 3600) + 1 for key in hashtags]))
num_windows_during = int(np.max([((end_unix_time - start_unix_time) // 3600 * 12) for key in hashtags]))
num_windows_after = int(np.max([((ltt - end_unix_time) // 3600) + 1 for key in hashtags]))


In [8]:
""" Organize data into specific time periods:
     before 2/1 8am with 1-hour windows, 
     between 2/1 8am and 2/1 8pm with 5-min windows,
     and after 2/1 8pm with 1-hour windows """

# Initialize dictionary for each time frame.
data_hashtag_before = {}
data_hashtag_during = {}
data_hashtag_after = {}

# Iterate through each hashtag.
for key in hashtags:
    print(key)
    
    # Rename the dictionary value for readability
    temp_before = hashtag_dict_before[key]
    temp_during = hashtag_dict_during[key]
    temp_after = hashtag_dict_after[key]
    
    data_hashtag_before[key] = np.zeros((num_windows_before, 5)) # Initialize array: rows = time window, columns = feature
    num_followers_before = {} # Initialize dictionary to count # of followers for each tweet
    
    
    
    # Iterate through all elements before start time
    for i in range(np.shape(temp_before)[0]):
        # Get row number
        item_before = int(num_windows_before - 1 - ((start_unix_time - temp_before[i,0] - 1) // 3600))
        # Update first 3 elements (# of tweets, total # retweets, total # followers)
        data_hashtag_before[key][item_before] += np.array([1, int(temp_before[i, 1]), int(temp_before[i, 2]), 0, 0])
        # Get time of day (hour)
        dt_obj_pst = datetime.fromtimestamp(temp_before[i,0], pst_tz)
        data_hashtag_before[key][item_before][4] = int(datetime.strftime(dt_obj_pst, '%H'))
        # Get number of followers
        if item_before not in num_followers_before.keys():
            num_followers_before[item_before] = []
        num_followers_before[item_before].append(temp_before[i,2])
    for i in num_followers_before.keys():
        data_hashtag_before[key][i][3] = np.max(num_followers_before[i])
        
        
    # Iterate through all elements during time
    data_hashtag_during[key] = np.zeros((num_windows_during, 5))
    num_followers_during = {}
    for i in range(np.shape(temp_during)[0]):
        item_during = int(((temp_during[i,0] - start_unix_time) * 12) // 3600)
        data_hashtag_during[key][item_during] += np.array([1, int(temp_during[i, 1]), int(temp_during[i, 2]), 0, 0])
        dt_obj_pst = datetime.fromtimestamp(temp_during[i,0], pst_tz)
        data_hashtag_during[key][item_during][4] = int(datetime.strftime(dt_obj_pst, '%H'))
        
        if item_during not in num_followers_during.keys():
            num_followers_during[item_during] = []
        num_followers_during[item_during].append(temp_during[i,2])
    for i in num_followers_during.keys():
        data_hashtag_during[key][i][3] = np.max(num_followers_during[i])
        
    # Iterate through all elements after end time
    data_hashtag_after[key] = np.zeros((num_windows_after, 5))
    num_followers_after = {}
    for i in range(np.shape(temp_after)[0]):
        item_after = int((temp_after[i,0] - end_unix_time) // 3600)
        data_hashtag_after[key][item_after] += np.array([1, int(temp_after[i, 1]), int(temp_after[i, 2]), 0, 0])
        dt_obj_pst = datetime.fromtimestamp(temp_after[i,0], pst_tz)
        data_hashtag_after[key][item_after][4] = int(datetime.strftime(dt_obj_pst, '%H'))
        
        if item_after not in num_followers_after.keys():
            num_followers_after[item_after] = []
        num_followers_after[item_after].append(temp_after[i,2])
    for i in num_followers_after.keys():
        data_hashtag_after[key][i][3] = np.max(num_followers_after[i])
        
print('done')

gohawks
gopatriots
nfl
patriots
sb49
superbowl
done


In [9]:
""" Aggregate data within each time period by combining all hashtags. """

# Initialize aggregated data variables
data_aggregate_before = np.zeros([num_windows_before, 5])
data_aggregate_during = np.zeros([num_windows_during, 5])
data_aggregate_after = np.zeros([num_windows_after, 5])

# Sum the # of tweets, total # of retweets, and # of followers
for key in hashtags:
    data_aggregate_before[:,0:3] += data_hashtag_before[key][:,0:3]
    data_aggregate_during[:,0:3] += data_hashtag_during[key][:,0:3]
    data_aggregate_after[:,0:3] += data_hashtag_after[key][:,0:3]
# Find the max # of followers for each
data_aggregate_before[:,3] = np.amax([data_hashtag_before[key][:,3] for key in hashtags], axis=0)
data_aggregate_during[:,3] = np.amax([data_hashtag_during[key][:,3] for key in hashtags], axis=0)
data_aggregate_after[:,3] = np.amax([data_hashtag_after[key][:,3] for key in hashtags], axis=0)

# Copy over the same time frames
data_aggregate_before[:,4] = data_hashtag_before['superbowl'][:,4]
data_aggregate_during[:,4] = data_hashtag_during['superbowl'][:,4]
data_aggregate_after[:,4] = data_hashtag_after['superbowl'][:,4]

In [10]:
""" Get data for the whole time frame with 1-hour windows, separated by hashtag """

# Initialize dictionary to store data.
# Key: hashtag
# Value: data separated by 1-hour time windows
data_hashtag_all = {}

for key in hashtags: # Iterate through all hashtags
    temp_during = np.zeros([12, 5]) # Initialize array to store data in the middle time period
    # Combine data in the middle time period
    for i in range(np.shape(data_hashtag_during[key])[0]):
        hour = int(data_hashtag_during[key][i,4] - 8)
        temp_during[hour, :3] += data_hashtag_during[key][i, :3]
        if not i % 12:
            temp_during[hour, 3] = np.max(data_hashtag_during[key][i:(i+12), 3])
            temp_during[hour, 4] = data_hashtag_during[key][i,4]
    data_hashtag_all[key] = np.vstack((data_hashtag_before[key], temp_during, data_hashtag_after[key]))

In [11]:
""" Combine data for the whole time frame from all hashtags """

data_all = np.zeros([587, 5])
for key in hashtags:
    data_all += data_hashtag_all[key]

### Neural Network

In [12]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [13]:
# Define layer sizes.
layer_sizes = [(50, 50), (100, 100), (100, 100, 100), (100, 100, 100, 100), 10*(50,), 10*(100,)]

In [19]:
# Function to get MSE for a given neural network model.

def analyze_nn(nn, X, y):
    mses_per_fold = []
    mses_per_fold_train = []
    kf = KFold(10)
    for trainset, testset in kf.split(X):
        X_train, y_train = X[trainset], y[trainset]
        X_test, y_test = X[testset], y[testset]
        nn.fit(X_train, y_train)
        predicted = nn.predict(X_test)
        mses_per_fold.append(mean_squared_error(y_test, predicted))
        mses_per_fold_train.append(mean_squared_error(y_train, nn.predict(X_train)))
    #     print(mean_squared_error(y_test, predicted))
    avg_mse = np.mean(mses_per_fold)
    avg_mse_train = np.mean(mses_per_fold_train)
    print('Layer size {} MSE:\n   val = {} \n train = {}'.format(size, np.around(avg_mse, 2), np.around(avg_mse_train, 2)))
    
    return avg_mse, avg_mse_train

#### No preprocessing

In [20]:
# Analyze tweets before start time.

# Define train data and targets
y_before = data_aggregate_before[1:,0] # Number of tweets (except first)
X_before = np.delete(data_aggregate_before, -1, 0) # Delete last row

X = X_before
y = y_before

print('X shape:', X.shape)
print('y shape:', y.shape)

mses_before = []
mses_before_train = []
for size in layer_sizes:
    nn = MLPRegressor(hidden_layer_sizes=size, activation='relu', solver='adam', alpha=0.001)

    avg_mse, avg_mse_train = analyze_nn(nn, X, y)
    mses_before.append(avg_mse)
    mses_before_train.append(avg_mse_train)

X shape: (439, 5)
y shape: (439,)
Layer size (50, 50) MSE:
   val = 1010218024.56 
 train = 1871794056.48
Layer size (100, 100) MSE:
   val = 36763868.01 
 train = 19924541.65
Layer size (100, 100, 100) MSE:
   val = 29750061.62 
 train = 13490066.19
Layer size (100, 100, 100, 100) MSE:
   val = 7393184.44 
 train = 7708099.22
Layer size (50, 50, 50, 50, 50, 50, 50, 50, 50, 50) MSE:
   val = 7137193.83 
 train = 6138574.94
Layer size (100, 100, 100, 100, 100, 100, 100, 100, 100, 100) MSE:
   val = 7150552.89 
 train = 6994886.77


In [23]:
y_during = data_aggregate_during[1:,0]
X_during = np.delete(data_aggregate_during, -1, 0)

X = X_during
y = y_during

print('X shape:', X.shape)
print('y shape:', y.shape)

mses_during = []
mses_during_train = []
for size in layer_sizes:
    nn = MLPRegressor(hidden_layer_sizes=size, activation='relu', solver='adam', alpha=0.001)

    avg_mse, avg_mse_train = analyze_nn(nn, X, y)
    mses_during.append(avg_mse)
    mses_during_train.append(avg_mse_train)

X shape: (143, 5)
y shape: (143,)
Layer size (50, 50) MSE:
   val = 2608589227753.65 
 train = 3034344389754.98
Layer size (100, 100) MSE:
   val = 710327656194.4 
 train = 1629857173993.65
Layer size (100, 100, 100) MSE:
   val = 260590650188.72 
 train = 155988861854.41
Layer size (100, 100, 100, 100) MSE:
   val = 531910790702.46 
 train = 245932473580.0
Layer size (50, 50, 50, 50, 50, 50, 50, 50, 50, 50) MSE:
   val = 413870891.51 
 train = 208691671.78
Layer size (100, 100, 100, 100, 100, 100, 100, 100, 100, 100) MSE:
   val = 81216414.96 
 train = 160468168.41


In [24]:
y_after = data_aggregate_after[1:,0]
X_after = np.delete(data_aggregate_after, -1, 0)

X = X_after
y = y_after

print('X shape:', X.shape)
print('y shape:', y.shape)

mses_after = []
mses_after_train = []
for size in layer_sizes:
    nn = MLPRegressor(hidden_layer_sizes=size, activation='relu', solver='adam', alpha=0.001)

    avg_mse, avg_mse_train = analyze_nn(nn, X, y)
    mses_after.append(avg_mse)
    mses_after_train.append(avg_mse_train)

X shape: (134, 5)
y shape: (134,)
Layer size (50, 50) MSE:
   val = 150161537155.33 
 train = 338168237422.7
Layer size (100, 100) MSE:
   val = 82339661067.84 
 train = 323818217335.24
Layer size (100, 100, 100) MSE:
   val = 97950083247.72 
 train = 75552631349.23
Layer size (100, 100, 100, 100) MSE:
   val = 935471492.37 
 train = 4974564658.31
Layer size (50, 50, 50, 50, 50, 50, 50, 50, 50, 50) MSE:
   val = 4110088.16 
 train = 7308543.41
Layer size (100, 100, 100, 100, 100, 100, 100, 100, 100, 100) MSE:
   val = 18510871.57 
 train = 77976263.19


In [25]:
print(np.sqrt(mses_before))
print(np.sqrt(mses_during))
print(np.sqrt(mses_after))

[31783.9271418   6063.3215324   5454.361706    2719.04108747
  2671.55270113  2674.05177323]
[1615112.76007394  842809.38307211  510480.80295808  729322.14466754
   20343.81703395    9012.01503341]
[387506.82207586 286948.88232548 312969.78008703  30585.47845584
   2027.3352355    4302.42624175]


#### Standard scaler preprocessing

In [27]:
from sklearn.preprocessing import StandardScaler

In [28]:
scaler = StandardScaler()

In [29]:
X = scaler.fit_transform(X_before) # Transform data
y = y_before

print('X shape:', X.shape)
print('y shape:', y.shape)

mses_before = []
mses_before_train = []
for size in layer_sizes:
    nn = MLPRegressor(hidden_layer_sizes=size, activation='relu', solver='adam', alpha=0.001)

    avg_mse, avg_mse_train = analyze_nn(nn, X, y)
    mses_before.append(avg_mse)
    mses_before_train.append(avg_mse_train)

X shape: (439, 5)
y shape: (439,)
Layer size (50, 50) MSE:
   val = 6148493.27 
 train = 5824639.53
Layer size (100, 100) MSE:
   val = 5255096.62 
 train = 4823395.97
Layer size (100, 100, 100) MSE:
   val = 4762487.76 
 train = 4394404.38
Layer size (100, 100, 100, 100) MSE:
   val = 4780836.03 
 train = 4094066.87
Layer size (50, 50, 50, 50, 50, 50, 50, 50, 50, 50) MSE:
   val = 5552782.69 
 train = 2091999.23
Layer size (100, 100, 100, 100, 100, 100, 100, 100, 100, 100) MSE:
   val = 6121224.55 
 train = 527643.78


In [30]:
X = scaler.fit_transform(X_during)
y = y_during

print('X shape:', X.shape)
print('y shape:', y.shape)

mses_during = []
mses_during_train = []
for size in layer_sizes:
    nn = MLPRegressor(hidden_layer_sizes=size, activation='relu', solver='adam', alpha=0.001)

    avg_mse, avg_mse_train = analyze_nn(nn, X, y)
    mses_during.append(avg_mse)
    mses_during_train.append(avg_mse_train)

X shape: (143, 5)
y shape: (143,)
Layer size (50, 50) MSE:
   val = 303735510.78 
 train = 297836889.64
Layer size (100, 100) MSE:
   val = 286588345.12 
 train = 281418918.9
Layer size (100, 100, 100) MSE:
   val = 74528289.47 
 train = 68155937.67
Layer size (100, 100, 100, 100) MSE:
   val = 55210953.1 
 train = 42493890.8
Layer size (50, 50, 50, 50, 50, 50, 50, 50, 50, 50) MSE:
   val = 31102708.12 
 train = 18156331.95
Layer size (100, 100, 100, 100, 100, 100, 100, 100, 100, 100) MSE:
   val = 25869895.37 
 train = 13202860.19


In [31]:
X = scaler.fit_transform(X_after)
y = y_after

print('X shape:', X.shape)
print('y shape:', y.shape)

mses_after = []
mses_after_train = []
for size in layer_sizes:
    nn = MLPRegressor(hidden_layer_sizes=size, activation='relu', solver='adam', alpha=0.001)

    avg_mse, avg_mse_train = analyze_nn(nn, X, y)
    mses_after.append(avg_mse)
    mses_after_train.append(avg_mse_train)

X shape: (134, 5)
y shape: (134,)
Layer size (50, 50) MSE:
   val = 5426468.68 
 train = 5439781.85
Layer size (100, 100) MSE:
   val = 3551484.09 
 train = 3687233.98
Layer size (100, 100, 100) MSE:
   val = 1663625.18 
 train = 806968.78
Layer size (100, 100, 100, 100) MSE:
   val = 1490343.65 
 train = 536685.99
Layer size (50, 50, 50, 50, 50, 50, 50, 50, 50, 50) MSE:
   val = 1118857.68 
 train = 320217.43
Layer size (100, 100, 100, 100, 100, 100, 100, 100, 100, 100) MSE:
   val = 801221.43 
 train = 119338.52


In [40]:
print(np.sqrt(mses_before))
print(np.sqrt(mses_during))
print(np.sqrt(mses_after))

[2459.72228665 2260.74253704 2179.77686376 2240.55712    2412.39575917
 2656.64054704]
[17428.35227655 16989.99993298  8969.17635395  7229.76899966
  5204.56281528  5284.81288082]
[2315.67987834 1996.51160876 1329.91335651 1200.58532616 1038.3718009
  936.02790447]
